In [108]:
import importlib
from chocodetector import *
chocodetector = importlib.import_module('chocodetector')
importlib.reload(chocodetector)

<module 'chocodetector' from '/home/hthums/master_projects/IAPR/project/project/chocodetector.py'>

In [32]:
ref_dir = 'chocolate-recognition-classic/dataset_project_iapr2025/references/'
train_dir = 'chocolate-recognition-classic/dataset_project_iapr2025/train/'

df_gt = pd.read_csv('chocolate-recognition-classic/dataset_project_iapr2025/train.csv')


df_pred = ChocoDetector(ref_path=ref_dir, train_path=train_dir, features=['gabor', 'lbp'],
                        pca_percentile=0.95, blob_percentile=95, window_size=80, step_size=10, ood_percentile=95, 
                        dt_gt=df_gt)

In [ ]:
y_true = df_gt.iloc[:, 1:].values
y_pred = df_pred.iloc[:, 1:].values

macro_f1, f1_per_class = f1_score_counts(y_true, y_pred)

print(f"Macro F1 Score: {macro_f1:.4f}")

for choc, f1_val in zip(df_gt.columns[1:], f1_per_class):
    print(f"{choc}: {f1_val:.4f}")

Macro F1 Score (count-based): 0.2085
Jelly White: 0.3357
Jelly Milk: 0.2000
Jelly Black: 0.2411
Amandina: 0.4133
Crème brulée: 0.2647
Triangolo: 0.1935
Tentation noir: 0.0000
Comtesse: 0.0333
Noblesse: 0.4478
Noir authentique: 0.2716
Passion au lait: 0.0000
Arabia: 0.1194
Stracciatella: 0.1905


In [110]:
# Load the reference images and training images

ref_dir = 'chocolate-recognition-classic/dataset_project_iapr2025/references/'
train_dir = 'chocolate-recognition-classic/dataset_project_iapr2025/train/'

ref_image_names = os.listdir(ref_dir)
ref_images = [cv2.imread(os.path.join(ref_dir, img)) for img in ref_image_names]
ref_images = [cv2.resize(img, (img.shape[1] // 4, img.shape[0] // 4)) for img in ref_images]
ref_images_rgb = [cv2.cvtColor(img, cv2.COLOR_BGR2RGB) for img in ref_images]
ref_images_gray = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in ref_images]

rgb_images, gray_images, train_names = load_training_images(train_dir)

features = ['gabor', 'lbp', 'hisogram']

df_gt = pd.read_csv('chocolate-recognition-classic/dataset_project_iapr2025/train.csv')

Optimization

In [113]:
def evaluate_detector(**kwargs):
    df_pred = ChocoDetectorv2(
        ref_images_rgb=ref_images_rgb,
        ref_images_gray=ref_images_gray,
        ref_image_names=ref_image_names,
        rgb_images=rgb_images,
        train_names=train_names,
        features=features,
        df_gt=df_gt,
        **kwargs
    )

    y_true = df_gt.iloc[:, 1:].values
    y_pred = df_pred.iloc[:, 1:].values

    macro_f1, _ = f1_score_counts(y_true, y_pred)
    return macro_f1

In [ ]:
# Define search ranges for each hyperparameter
pca_range = np.array([0.95, 0.9999])              # Step 1
blob_range = np.arange(95, 99, 1)                     # Step 2
window_range = np.arange(55, 65, 1)                  # Step 3
step_range = np.arange(8, 12, 1)                       # Step 4
ood_range = np.arange(50, 100, 10)                      # Step 5

# Initialize default/baseline values
best_params = {
    "pca_percentile": 0.95,
    "blob_percentile": 95,
    "window_size": 80,
    "step_size": 10,
    "ood_percentile": 95
}

# Step 1: Optimize PCA percentile
best_score = -1
for pca in pca_range:
    params = {**best_params, "pca_percentile": pca}
    score = evaluate_detector(**params)
    if score > best_score:
        best_score = score
        best_params["pca_percentile"] = pca
print(f"Best PCA: {best_params['pca_percentile']}, Score: {best_score:.4f}")

# Step 2: Optimize blob percentile
best_score = -1
for blob in blob_range:
    params = {**best_params, "blob_percentile": blob}
    score = evaluate_detector(**params)
    if score > best_score:
        best_score = score
        best_params["blob_percentile"] = blob
print(f"Best Blob: {best_params['blob_percentile']}, Score: {best_score:.4f}")

# Step 3: Optimize window size
best_score = -1
for window in window_range:
    params = {**best_params, "window_size": window}
    score = evaluate_detector(**params)
    if score > best_score:
        best_score = score
        best_params["window_size"] = window
print(f"Best Window: {best_params['window_size']}, Score: {best_score:.4f}")

# Step 4: Optimize step size
best_score = -1
for step in step_range:
    params = {**best_params, "step_size": step}
    score = evaluate_detector(**params)
    if score > best_score:
        best_score = score
        best_params["step_size"] = step
print(f"Best Step: {best_params['step_size']}, Score: {best_score:.4f}")

# Step 5: Optimize OOD percentile
best_score = -1
for ood in ood_range:
    params = {**best_params, "ood_percentile": ood}
    score = evaluate_detector(**params)
    if score > best_score:
        best_score = score 
        best_params["ood_percentile"] = ood
print(f"Best OOD: {best_params['ood_percentile']}, Score: {best_score:.4f}")


In [ ]:
best_params = {
    "pca_percentile": 0.95,
    "blob_percentile": 95,
    "window_size": 60,
    "step_size": 10,
    "ood_percentile": 95
}

print(evaluate_detector(**best_params))